In [1]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

df = pd.read_excel('kerevit_mod_final.xlsx')

X = df[['W', 'KB', 'KE', 'AB', 'AE', 'K_Sag', 'K_Sol', 'U_Sag', 'U_Sol', 'KE_Sag', 'KE_Sol']]
y = df['CINSIYET']

pipe = Pipeline([
    ('scaler', StandardScaler()),  
    ('knn', KNeighborsClassifier())  
])

param_grid = {'knn__n_neighbors': list(range(1, 51, 2))}

grid = GridSearchCV(pipe, param_grid, cv=10, n_jobs=-1, scoring='accuracy')
grid.fit(X, y)

print("En iyi parametreler: ", grid.best_params_)
print("En iyi doğruluk: ", grid.best_score_)

C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


En iyi parametreler:  {'knn__n_neighbors': 7}
En iyi doğruluk:  0.7212121212121213


# Eğitim ve Test Aşaması

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import joblib

In [3]:
df = pd.read_excel('kerevit_mod_final.xlsx')
X = df[['W', 'KB', 'KE', 'AB', 'AE', 'K_Sag', 'K_Sol', 'U_Sag', 'U_Sol', 'KE_Sag', 'KE_Sol']]
y = df['CINSIYET']
sira = df['Sira']  # Sıra bilgisini almak

kf = KFold(n_splits=10, shuffle=False)
conf_matrices = []
fold = 1

results = pd.DataFrame(columns=['Sira', 'Tahmin', 'Gerçek'])

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    sira_test = sira.iloc[test_index]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    knn = KNeighborsClassifier(n_neighbors=7)
    knn.fit(X_train, y_train)
    
    model_filename = f'knn_model_fold_{fold}.joblib'
    joblib.dump(knn, model_filename)
    
    y_pred = knn.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    conf_matrices.append(cm)
    
    print(f"Fold {fold}:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", cm)
    print("\n")
    
    # Sonuçları toplama
    fold_results = pd.DataFrame({
        'Sira': sira_test,
        'Tahmin': y_pred,
        'Gerçek': y_test
    })
    results = pd.concat([results, fold_results])
    
    fold += 1

total_conf_matrix = np.zeros((2, 2), dtype=int)  
for i, cm in enumerate(conf_matrices, 1):
    print(f"Confusion Matrix for Fold {i}:\n{cm}\n")
    total_conf_matrix += cm

print("Total Confusion Matrix:\n", total_conf_matrix)

TN, FP, FN, TP = total_conf_matrix.ravel()

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f"Overall Accuracy: {accuracy:.4f}")
print(f"Overall Precision: {precision:.4f}")
print(f"Overall Recall: {recall:.4f}")
print(f"Overall Specificity: {specificity:.4f}")
print(f"Overall F1 Score: {f1_score:.4f}")

results.to_excel('knn_predict_mod.xlsx', index=False)

Fold 1:
Accuracy: 0.9166666666666666
Classification Report:
               precision    recall  f1-score   support

           D       0.88      1.00      0.93         7
           E       1.00      0.80      0.89         5

    accuracy                           0.92        12
   macro avg       0.94      0.90      0.91        12
weighted avg       0.93      0.92      0.91        12

Confusion Matrix:
 [[7 0]
 [1 4]]


Fold 2:
Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

           D       0.75      0.86      0.80         7
           E       0.75      0.60      0.67         5

    accuracy                           0.75        12
   macro avg       0.75      0.73      0.73        12
weighted avg       0.75      0.75      0.74        12

Confusion Matrix:
 [[6 1]
 [2 3]]


Fold 3:
Accuracy: 0.7272727272727273
Classification Report:
               precision    recall  f1-score   support

           D       0.71      0.83      0.77       

C:\Users\Yasin\AppData\Local\Temp/ipykernel_10920/3662047262.py:69: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  results.to_excel('knn_predict_mod.xlsx', index=False)
